In [16]:
import numpy
import skrf

In [24]:
def thru2x(thru_path, *args, **kwargs):
        if 'export_path' in kwargs:
            export_path = kwargs['export_path']
        else:
            export_path = '.\\'

        thru =  skrf.Network(thru_path)
        
        f_points = thru.frequency.npoints
        f_list   = thru.frequency

        s_fxr = numpy.zeros([f_points,2,2],dtype=numpy.complex128)
        s11_fxr = numpy.array([])
        s21_fxr = numpy.array([])
        s21_phase = numpy.array([])

        for i in range(0,f_points):
            s11_t = numpy.complex128(thru.s[i][0][0])
            s12_t = numpy.complex128(thru.s[i][0][1])
            s21_t = numpy.complex128(thru.s[i][1][0])
            s22_t = numpy.complex128(thru.s[i][1][1])

            s11_fxr = numpy.append(s11_fxr, numpy.divide((s11_t + s22_t), (2 + s12_t + s21_t)))        # Ref. (5) in [1]
            s21_fxr = numpy.append(s21_fxr, numpy.sqrt(0.5 * (s12_t + s21_t) * (1 - s11_fxr**2)))      # Ref. (6) in [1]
            s21_phase = numpy.append(s21_phase, numpy.angle(s21_fxr[i]) * 180/numpy.pi)

            # s12_fxr = s21_fxr
            # s22_fxr = s11_fxr
            # s_fxr[i] = [[s11_fxr, s12_fxr],[s21_fxr,s22_fxr]]
        pass

        x = s21_phase[1:len(s21_phase)] - s21_phase[0:len(s21_phase)-1]
        a = numpy.array([])
        for j in range(0, len(x)):
            a = numpy.append(a, ((x[j]<-90)*360+(x[j]>90)*(-360)))
        # a = numpy.cumsum(a)
        # a = numpy.insert(a, 0, 0)
        # s21_phase_unwrapped = s21_phase + a
        
        # print(a)
        print(s21_phase)


y = thru2x(r'D:\RF_Microwave_Library\de_embedding\cal_std\sim\thru.s2p', export_path=r'D:\RF_Microwave_Library\de_embedding\cal_std\sim') 

[-33.64945775 -50.5369095  -61.1081054  -66.72467495 -77.29587085
 -73.50277974 -82.2848298   87.1439743  -89.06293459 -80.52172898
  85.2385522   74.6673563   78.46044741  87.00165302 -85.2525837
  76.0920652   65.5208693   69.31396041  77.85516602  85.6009293
 -88.82359345  69.2770147   58.7058188   62.49890991  71.04011552
  78.7858788   84.36135605  88.27103292  63.9414272   53.3702313
  57.16332241  65.70452802  73.4502913   79.02576855  82.93544542
  85.78307553  59.5631847   48.9919888   52.78507991  61.32628552
  69.0720488   74.64752605  78.55720292  81.40483303  83.57451436
  55.8365267   45.2653308   49.05842191  57.59962752  65.3453908
  70.92086805  74.83054492  77.67817503  79.84785636  81.56238823
  52.5772932   42.0060973   45.79918841  54.34039402  62.0861573
  67.66163455  71.57131142  74.41894153  76.58862286  78.30315473
  79.69365107  49.6691767   39.0979808   42.89107191  51.43227752
  59.1780408   64.75351805  68.66319492  71.51082503  73.68050636
  75.39503823  